# Regularised models

# Two aspects of regularisation:
## - TV loss on the reconstruction
## - L1 loss on the weights of the bottleneck layer

In [1]:
import matplotlib.pyplot as plt
import netCDF4
import numpy as np
import pandas as pd
from mpl_toolkits.basemap import Basemap
from tqdm import tqdm
###################################################
import torch
from sklearn.decomposition import IncrementalPCA, PCA
from sklearn.model_selection import train_test_split
###################################################
import torch
import torchvision as tv
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
###################################################
from livelossplot import PlotLosses
import hiddenlayer as hl
###################################################
import pickle
import os
###################################################
import scipy as sp
import scipy.fftpack

In [2]:
temp_nc = netCDF4.Dataset('../eniko/data/tas/tas_day_MPI-ESM-MR_rcp85_r1i1p1_g025.nc')
temp_ncdata_3D = np.array(temp_nc.variables['tas'])
temp_ncdata_2D = temp_ncdata_3D.reshape(temp_ncdata_3D.shape[0], temp_ncdata_3D.shape[1]*temp_ncdata_3D.shape[2])        

dates = pd.to_datetime(netCDF4.num2date(temp_nc.variables['time'][:], temp_nc.variables['time'].units)).year
temp_ncdata_df = pd.DataFrame(temp_ncdata_2D, index = dates)

In [3]:
temp_ncdata_df.shape

(84371, 10368)

In [4]:
train_start = 2025
train_end = 2100

test_start = 1975
test_end = 2000

max_fts = 10
batch_size = 32

In [5]:
# PCA type
train_idx = temp_ncdata_df.index.isin(range(train_start,train_end+1))
train_samples = temp_ncdata_df.iloc[train_idx]
train_years = temp_ncdata_df.index[temp_ncdata_df.index.isin(range(train_start,train_end+1))]

test_idx = temp_ncdata_df.index.isin(range(test_start,test_end+1))
test_samples = temp_ncdata_df.iloc[test_idx]
test_years = temp_ncdata_df.index[temp_ncdata_df.index.isin(range(test_start,test_end+1))]

# Standardise the training samples
train_samples = torch.tensor(train_samples.values)#.double()
test_samples = torch.tensor(test_samples.values)#.double()

mu, sig = train_samples.mean(), train_samples.std()

# train_samples = train_samples.sub(mu).div(sig)
# test_samples = test_samples.sub(mu).div(sig)

mmin = min(train_samples.min(),test_samples.min())
mmax = max(train_samples.max(),test_samples.max())

# # Normalise instead
train_samples = (train_samples - mmin) / (mmax - mmin)
test_samples = (test_samples - mmin) / (mmax - mmin)


train_data = DataLoader(train_samples, batch_size=batch_size)
test_data = DataLoader(test_samples, batch_size=batch_size)

In [6]:
# AE type (reshaped)
train_data_ae = DataLoader(train_samples.reshape(-1, 72, 144), batch_size=batch_size)
test_data_ae = DataLoader(test_samples.reshape(-1, 72, 144), batch_size=batch_size)

In [7]:
dataloaders = {
    "train": train_data_ae,
    "validation": test_data_ae
}

In [8]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [74]:
class AutoEncoder(nn.Module):
    def __init__(self, layers, TV_norm=False, L1_norm=False):
        super(AutoEncoder,self).__init__()
        assert len(layers) == 5, 'Not the right number of layer numbers'
        self.encoder = nn.Sequential(
            nn.Conv2d(1, layers[0], kernel_size=(3,5))
            ,nn.MaxPool2d(2)
            ,nn.ReLU()
            
            ,nn.Conv2d(layers[0], layers[1], kernel_size=(2,5))
            ,nn.MaxPool2d((2,3))
            ,nn.ReLU()
            
            ,nn.Conv2d(layers[1], layers[2], kernel_size=(2,3))
            ,nn.MaxPool2d(2)
            ,nn.ReLU()
            
            ,nn.Conv2d(layers[2], layers[3], kernel_size=(3,3))
            ,nn.MaxPool2d(2)
            ,nn.ReLU()
            
            ,nn.Conv2d(layers[3], layers[4], kernel_size=(2,3), bias=~L1_norm) # Bias included if L1_norm not needed
            ,nn.MaxPool2d(2)
        )
        
        self.decoder = nn.Sequential(
            
            nn.ConvTranspose2d(layers[4], layers[3], kernel_size=(3,4), stride=(3,4))
            ,nn.ReLU()
                    
            ,nn.ConvTranspose2d(layers[3], layers[2], kernel_size=(3,3), stride=(3,3))
            ,nn.ReLU()
            
            ,nn.ConvTranspose2d(layers[2], layers[1], kernel_size=(2,3), stride=(2,3))
            ,nn.ReLU()
            
#             ,nn.PixelShuffle(2)
            
            ,nn.ConvTranspose2d(layers[1], layers[0], kernel_size=2, stride=2)
            ,nn.ReLU()
                       
            ,nn.ConvTranspose2d(layers[0], 1, kernel_size=(2,2), stride=(2,2))
            ,nn.Sigmoid()
        )
        
    def forward(self, x):
        # Encoder part
        y = self.encoder(x)
        # Decoder part
        x = self.decoder(y)
        return y, x # x is the output of the autoencoder, y is the output of the encoder

In [75]:
def train_model(model, criterion, optimizer, bottleneck, num_epochs=20, TV_norm=False, L1_norm=False):
    liveloss = PlotLosses(skip_first=2
                          ,max_cols=1
                          ,fig_path='data/images/autoencoder/TV_norm_{}_L1_norm_{}_train_test_loss_{}_lambda_tv_{}_lambda_l1_{}.pdf'.format(\
                                                                str(TV_norm)
                                                                ,str(L1_norm)
                                                                ,bottleneck
                                                                ,lambda_tv
                                                                ,lambda_l1))
    model = model.to(device)
    
    for epoch in range(num_epochs):
        logs = {}
        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0

            for inputs in dataloaders[phase]:
                
                inputs = inputs.to(device).unsqueeze(1)
                encoded, decoded = model(inputs)
                
                loss = criterion(inputs, decoded)
#                 print('loss is ' + str(loss.detach()))
                if TV_norm:
                    # Create the TV loss on the decoded image
                    # Horizontal direction including over the 'vertical edge'
                    tv_loss = torch.mean(torch.abs(torch.cat(\
                                (decoded[:, :, :, :-1] - decoded[:, :, :, 1:]\
                                ,(decoded[:, :, :, -1] - decoded[:, :, :, 0]).unsqueeze(-1)), dim=-1)))
                    
                    # Vertical direction loss - add 2 * since the aspect ratio of horizontal to vertical is 2:1
                    # So to even out the derivatives in each direction, multiply this by 2
                    tv_loss += 2 * torch.mean(torch.abs(decoded[:, :, :-1, :] - decoded[:, :, 1:, :]))
#                     print('tv loss is ' + str(lambda_tv*tv_loss.detach()))
                    loss += lambda_tv * tv_loss
                
                if L1_norm:
                    bn_weights = model.encoder[12].weight.detach()
                    # Create the bottleneck loss on the weights of the bottleneck layer (this is the max pool layer?)
                    l1_loss = lambda_l1 * torch.norm(bn_weights, 1)
                    
                    loss += l1_loss
                        
                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                running_loss += loss.detach() * inputs.size(0)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            
            prefix = ''
            if phase == 'validation':
                prefix = 'val_'

            logs[prefix + 'MSE + reg loss'] = epoch_loss.item()
        
        liveloss.update(logs)
        liveloss.draw()
    
    return model

In [76]:
# This will train the model
num_epochs = 3
bottleneck= 256

# Parameters for regularisation - what should these be??
lambda_tv = 0.02
lambda_l1 = 1.0

TV_norm = True
L1_norm = False

# Initialise AE
model = AutoEncoder([8, 32, 64, 128, 256], TV_norm=TV_norm, L1_norm=L1_norm)
optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)
criterion = nn.MSELoss()

# Start training
trained_model = train_model(model, criterion, optimizer, bottleneck, num_epochs, TV_norm, L1_norm)

In [80]:
model_name = 'data/models/autoencoder/ae_reg_TV_norm_{}_L1_norm_{}_{}_{}_to_{}_lambda_tv_{}_lambda_l1_{}.pickle'.format(str(TV_norm)
                                                                                                ,str(L1_norm)
                                                                                                ,bottleneck
                                                                                                ,train_start
                                                                                                ,train_end
                                                                                                ,lambda_tv
                                                                                                ,lambda_l1)

os.makedirs(os.path.dirname(model_name), exist_ok=True)

torch.save(trained_model.state_dict(), model_name)